# YOLOv5 Training and Detection

Этот блокнот предназначен для обучения модели YOLOv5 и использования ее для распознавания изображений.

## 1. Обучение модели

In [1]:
# Импорт необходимых библиотек
import os
import cv2
import numpy as np

In [2]:
# Определение параметров обучения
img_size = 640  # размер изображения
batch_size = 16  # размер пакета
epochs = 50  # количество эпох
data_config = os.path.join('yolov5', 'config.yaml')  # путь к вашему .yaml файлу
weights = 'yolov5s.pt'  # предобученные веса

# Запуск процесса обучения
# os.system(f'python train.py --img {img_size} --batch {batch_size} --epochs {epochs} --data {data_config} --weights {weights}')

# Запуск процесса обучения без размера изображения
os.system(f'python train.py --batch {batch_size} --epochs {epochs} --data {data_config} --weights {weights}')

2

## 2. Распознавание изображения на обученной модели

In [ ]:
# Определение параметров для распознавания
detect_weights = os.path.join('yolov5', 'runs', 'train', 'exp', 'weights', 'best.pt') # путь к оптимальным весам после обучения
conf_threshold = 0.25  # порог уверенности
image_source = os.path.join('path_to_your_image.jpg')  # путь к вашему изображению

# Запуск распознавания
# os.system(f'python detect.py --weights {detect_weights} --img {img_size} --conf {conf_threshold} --source {image_source}')

# Запуск распознавания без размера изображения
os.system(f'python detect.py --weights {detect_weights} --conf {conf_threshold} --source {image_source}')

# Проверка наличия результатов
result_directory = os.path.join('yolov5', 'runs', 'detect', 'exp') # Папка с результатами детекции
results_found = False  # Переменная для отслеживания наличия результатов

# Проверка, существуют ли обнаруженные картинки и соответствующие метки
for filename in os.listdir(result_directory):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Загрузка изображения
        image_path = os.path.join(result_directory, filename)
        img = cv2.imread(image_path)

        # Проверяем, найдено ли хотя бы одно обнаруженное лицо
        # Посмотрим наличие меток (например, если вы обнаружаете паспорта, паспорт должен иметь класс 0)
        label_file_path = os.path.join(result_directory, filename.replace('.jpg', '.txt').replace('.png', '.txt'))
        if os.path.exists(label_file_path):
            with open(label_file_path, 'r') as f:
                labels = f.readlines()
                if any('0' in label for label in labels):  # Проверка наличия класса '0'
                    results_found = True

# Вывод сообщения на экран
if results_found:
    print("Паспорт найден на изображении.")
else:
    print("Паспорт не найден на изображении.")

FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: 'runs\\detect\\exp'